In [1]:
import calendar
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import locale
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules,apriori
from scipy import sparse

In [2]:
reviewer = pd.read_csv('00. Review.csv')


reviewer.head(3)



,Product,UserName,SkinCond_Age,Recommend,PostDate,Review,Rating
0,Perfect 3D Gel,MALA_,35 - 39,MALA_ recommends this product!,12 Jul 2020,Di aku beneran kerja. Melembabkan bgt. Tp gak ...,5
1,Perfect 3D Gel,Vitamaylinda,"Normal, 19 - 24",Vitamaylinda doesn't recommend this product!,31 May 2020,ga cocok bgttt bikin muka bruntusan dan lgsg m...,2
2,Perfect 3D Gel,aishdwie,"Combination, 19 - 24",aishdwie doesn't recommend this product!,30 Jul 2020,[SOLD]baru pertama kali nyoba dan beli karna l...,2


In [3]:
product = pd.read_csv('00. InfoProduct.csv')

product.head(3)

,Category,Merk,Product,Price,OverallRating,Reviewer
0,Night Cream,Hada Labo,Perfect 3D Gel,Rp. 91.000,3.8,978.0
1,Face Mist,PIXY,Aqua Beauty Protecting Mist,Rp. 27.000,4.2,3851.0
2,Night Cream,Wardah,Lightening Night Cream,Rp. 0,3.5,806.0


In [4]:
df = pd.merge(reviewer,product,on='Product')

df = df.drop(columns=['Reviewer','Recommend','Review'])

df.head()
# df = df.sample(n=6560)

,Product,UserName,SkinCond_Age,PostDate,Rating,Category,Merk,Price,OverallRating
0,Perfect 3D Gel,MALA_,35 - 39,12 Jul 2020,5,Night Cream,Hada Labo,Rp. 91.000,3.8
1,Perfect 3D Gel,Vitamaylinda,"Normal, 19 - 24",31 May 2020,2,Night Cream,Hada Labo,Rp. 91.000,3.8
2,Perfect 3D Gel,aishdwie,"Combination, 19 - 24",30 Jul 2020,2,Night Cream,Hada Labo,Rp. 91.000,3.8
3,Perfect 3D Gel,ilmisaptiah,"Dry, 30 - 34",12 Jul 2020,3,Night Cream,Hada Labo,Rp. 91.000,3.8
4,Perfect 3D Gel,alvinadin,"Oily, 19 - 24",15 Jul 2020,3,Night Cream,Hada Labo,Rp. 91.000,3.8


In [5]:
df['UserName'].value_counts()

moonchild_dby    92
honaleva         59
Yunitaella       57
gurlbyby         41
nysrinaw         40
                 ..
ayi16             1
Khasbiana         1
Afiyafia          1
Nailaaz           1
heyduckoff        1
Name: UserName, Length: 5247, dtype: int64

In [6]:
df['Product'].value_counts()

Milk Cleanser                      360
Face Mist                          280
Pore Pack                          250
Real Nature Mask Sheet             250
Masquerade Face Mask               250
                                  ... 
Lipocils Platinum Eyelash Serum      2
Optibrow Enhancing Serum             2
A-Lash Wink Serum                    1
DHC 3 in 1 eyelash serum             1
Miss Thique Brow and Lash Serum      1
Name: Product, Length: 797, dtype: int64

In [7]:
df['Rating'].value_counts()

5    4509
4    3308
3    1903
2     741
1     365
Name: Rating, dtype: int64

In [8]:
df['PostDate'].value_counts()

a day ago      442
2 days ago     401
3 days ago     333
6 days ago     307
5 days ago     299
              ... 
25 Nov 2019      1
21 Sep 2017      1
01 Mar 2018      1
10 May 2018      1
18 Jun 2019      1
Name: PostDate, Length: 895, dtype: int64

In [9]:
df.isna().sum()


Product           0
UserName          0
SkinCond_Age     89
PostDate          0
Rating            0
Category          0
Merk              0
Price            50
OverallRating     0
dtype: int64

In [10]:
# df = df.dropna()

In [11]:
df.Product.value_counts()

Milk Cleanser                      360
Face Mist                          280
Pore Pack                          250
Real Nature Mask Sheet             250
Masquerade Face Mask               250
                                  ... 
Lipocils Platinum Eyelash Serum      2
Optibrow Enhancing Serum             2
A-Lash Wink Serum                    1
DHC 3 in 1 eyelash serum             1
Miss Thique Brow and Lash Serum      1
Name: Product, Length: 797, dtype: int64

In [12]:
df.Rating.value_counts()

5    4509
4    3308
3    1903
2     741
1     365
Name: Rating, dtype: int64

In [13]:
# df = df[df['Rating'] >= 2]

In [14]:
counts = df['UserName'].value_counts()

to_remove = counts[counts < 20].index.tolist()

df.drop(df[df['UserName'].isin(to_remove)].index, inplace=True)


In [15]:
item_count_pivot = df.pivot_table(index='UserName',columns='Product', values= 'Rating',aggfunc='sum').fillna(0)
item_count_pivot[item_count_pivot > 0] =1
item_count_pivot.head(10)

Product,7 Days Mask,99% Jeju Fresh Aloe Soothing Gel,Acne Blemish Cream,Acne Control Apricot Scrub,Acne Gel,Acne Lotion,Acne Patch,Acne Pimple Medication,Acne Solution AntiAcne Facial Foam,Acnederm Acne Spot Treatment Gel,...,White Perfect Clinical Overnight Treatment,White Plus Renew Capsule Sleeping Pack,White Secret Pure Treatment Essence,Wine Peeling Jelly Softener,Youthmud Tinglexfoliate Treatment,Yuja Niacin Serum,aloe vera soothing gel 95%,hutmun gel,real nature mask sheet,wardah aloe vera multifunction gel
UserName,,,,,,,,,,,,,,,,,,,,,
Cresencindy,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fitriayden,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ikara,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
Ilyy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Niaaja,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Yunitaella,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zulfaa_Kh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ajoew31,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
chintyachristina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Keterangan Olah Dataset
print('Dimensi Dataset :',item_count_pivot.shape)
print('Jumlah Reviewer Item :',item_count_pivot.shape[0])
print('Jumlah Item :',item_count_pivot.shape[1])

Dimensi Dataset : (36, 251)
Jumlah Reviewer Item : 36
Jumlah Item : 251


In [19]:
freq = apriori(item_count_pivot, min_support= 0.08, use_colnames= True)
freq.sort_values('support',ascending=False)

c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
15,0.277778,(Pore Pack)
6,0.277778,(Face Mist)
12,0.250000,(Milk Cleanser)
7,0.222222,(Face Tonic)
11,0.194444,(Masquerade Face Mask)
14,0.194444,(My Real Squeeze Mask)
28,0.166667,"(Face Tonic, Milk Cleanser)"
8,0.166667,(Instabright Tone Up Milk Mask)
17,0.166667,(Real Nature Mask Sheet)
21,0.138889,(Water Sleeping Mask)


In [20]:
rules = association_rules(freq,metric='lift',min_threshold=1)[['antecedents','consequents','support','confidence','lift']]
rules.sort_values(by=['support','confidence','lift'],ascending=False)

,antecedents,consequents,support,confidence,lift
8,(Face Tonic),(Milk Cleanser),0.166667,0.750000,3.000000
9,(Milk Cleanser),(Face Tonic),0.166667,0.666667,3.000000
11,(Face Tonic),(Pore Pack),0.111111,0.500000,1.800000
10,(Pore Pack),(Face Tonic),0.111111,0.400000,1.800000
34,(wardah aloe vera multifunction gel),"(Face Tonic, Milk Cleanser)",0.083333,1.000000,6.000000
5,(Apricot Jam Face Scrub),(Masquerade Face Mask),0.083333,1.000000,5.142857
13,(wardah aloe vera multifunction gel),(Face Tonic),0.083333,1.000000,4.500000
25,"(Pore Pack, Milk Cleanser)",(Face Tonic),0.083333,1.000000,4.500000
32,"(Milk Cleanser, wardah aloe vera multifunction...",(Face Tonic),0.083333,1.000000,4.500000
21,(wardah aloe vera multifunction gel),(Milk Cleanser),0.083333,1.000000,4.000000
